# Batch Daily Inference Pipeline
* Retrieve best model from Hopsworks
* Scrape weather forecasts for next 14 days
* Predict weather code and store predictions in Hopsworks

In [1]:
import hopsworks
import joblib
import numpy as np
import pandas as pd
from weather_utils import *

import sys
sys.path.append('..')  # Add the parent directory (project root) to the Python path
from config import *

# Disable annoying warnings
import warnings
warnings.filterwarnings("ignore")

# 1) Retrieve best model from Hopsworks

In [2]:
# Connect to Hopsworks
project = hopsworks.login()

# Retrieve Feature Store
fs = project.get_feature_store()

# Get Model Registry
mr = project.get_model_registry()

# Select best model based on evaluation metric
weather_code_model = mr.get_best_model(MODEL_NAME,
                          MODEL_METRIC,
                          OPTIMIZE_DIRECTION)

# Download model path
model_dir = weather_code_model.download()

# Download model
model = joblib.load(model_dir + '/'+ MODEL_NAME + '.pkl')

print("Model:", model_dir)

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/178324
Connected. Call `.close()` to terminate connection gracefully.
Connected. Call `.close()` to terminate connection gracefully.


# 2) Scrape weather forecasts for next 14 days

In [3]:
# Define query parameters
params = {
    "latitude": LATITUDE,
    "longitude": LONGITUDE,
    "daily": ["temperature_2m_min", "precipitation_sum", "wind_gusts_10m_max"],
    "timezone": TIMEZONE,
    "past_days": 0,
    "forecast_days": 14
}

# Setup connection with Open-Meteo
openmeteo = get_openmeteo_connection()

# Execute the query
responses = openmeteo.weather_api(BASELINE_URL_OPEN_METEO, params=params)

# Extract the location because the response can be done for multiple cities as well
response = responses[0]

# Process yesterday's data into a Pandas dataframe
df_forecasts = process_forecast_request(response)

display(df_forecasts)

date  temperature_min  precipitation_sum  wind_gusts_max
0   2023-11-15          -1.3825           0.000000       25.919998
1   2023-11-16          -0.7825           0.000000       25.199999
2   2023-11-17          -3.7325           0.000000       19.799999
3   2023-11-18          -4.7305           0.000000       13.320000
4   2023-11-19          -0.3305           0.000000       16.199999
5   2023-11-20          -2.1455           0.000000       18.359999
6   2023-11-21          -2.9455           7.500000       58.320000
7   2023-11-22           0.9045           5.399998       66.239998
8   2023-11-23          -6.3915           4.800000       32.039997
9   2023-11-24          -5.8415           0.900000       23.039999
10  2023-11-25          -2.4915           0.000000       33.480000
11  2023-11-26          -2.5915           0.000000       57.239998
12  2023-11-27           1.6085          10.500001       70.199997
13  2023-11-28           1.3085           1.800000       24.480000

# 3) Predict Weather Code

In [4]:
# Add a new column with the month as an integer
df_forecasts['month'] = pd.to_datetime(df_forecasts['date']).dt.month

# Select features for model
X = df_forecasts.drop(columns=["date"])
print(X.columns)

Index(['temperature_min', 'precipitation_sum', 'wind_gusts_max', 'month'], dtype='object')


In [5]:
# Train model on the training set
y = model.predict(X)

# Round predicted value to closest weather code
y = np.round(y).astype(int)

df_forecasts['weather_code'] = y

In [6]:
# Add weather code descriptions
df_forecasts = add_weather_code_labels(df_forecasts)

display(df_forecasts)

date  temperature_min  precipitation_sum  wind_gusts_max  month  \
0   2023-11-15          -1.3825           0.000000       25.919998     11   
1   2023-11-16          -0.7825           0.000000       25.199999     11   
2   2023-11-17          -3.7325           0.000000       19.799999     11   
3   2023-11-18          -4.7305           0.000000       13.320000     11   
4   2023-11-19          -0.3305           0.000000       16.199999     11   
5   2023-11-20          -2.1455           0.000000       18.359999     11   
6   2023-11-21          -2.9455           7.500000       58.320000     11   
7   2023-11-22           0.9045           5.399998       66.239998     11   
8   2023-11-23          -6.3915           4.800000       32.039997     11   
9   2023-11-24          -5.8415           0.900000       23.039999     11   
10  2023-11-25          -2.4915           0.000000       33.480000     11   
11  2023-11-26          -2.5915           0.000000       57.239998     11   
12  2023-11-27           1.6085          10.500001       70.199997     11   
13  2023-11-28           1.3085           1.800000       24.480000     11   

    weather_code                              weather_code_desc  \
0              2      Mainly clear, partly cloudy, and overcast   
1              2      Mainly clear, partly cloudy, and overcast   
2              2      Mainly clear, partly cloudy, and overcast   
3              2      Mainly clear, partly cloudy, and overcast   
4              2      Mainly clear, partly cloudy, and overcast   
5              2      Mainly clear, partly cloudy, and overcast   
6              6     Rain: Slight, moderate and heavy intensity   
7              5    Freezing Drizzle: Light and dense intensity   
8              6     Rain: Slight, moderate and heavy intensity   
9              4  Drizzle: Light, moderate, and dense intensity   
10             2      Mainly clear, partly cloudy, and overcast   
11             2      Mainly clear, partly cloudy, and overcast   
12             6     Rain: Slight, moderate and heavy intensity   
13             4  Drizzle: Light, moderate, and dense intensity   

   weather_code_desc_short  
0                    Clear  
1                    Clear  
2                    Clear  
3                    Clear  
4                    Clear  
5                    Clear  
6                     Rain  
7                  Drizzle  
8                     Rain  
9                  Drizzle  
10                   Clear  
11                   Clear  
12                    Rain  
13                 Drizzle

# 4) Update forecasts in Hopsworks Feature Group

In [7]:
# Get or create Feature Group
forecast_weather_fg = fs.get_or_create_feature_group(
    name=FG_FORECAST_NAME,
    version=FG_FORECAST_V,
    primary_key=FG_FORECAST_PK,
    event_time=["date"],
    description=FG_FORECAST_DESC,
    statistics_config={"enabled": True,
                       "histograms": True,
                       "correlations": True}
)

# Upload data
forecast_weather_fg.insert(df_forecasts,
                           write_options={"wait_for_job" : True})

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/178324/fs/178243/fg/226673


Uploading Dataframe: 0.00% |          | Rows 0/14 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: weather_forecast_fg_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/178324/jobs/named/weather_forecast_fg_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x1457f0210>, None)

In [8]:
# save forecast locally
df_forecasts.to_csv('../resources/forecast.csv')

In [9]:
# upload forecast to Hopsworks cluster for Hugging Face
dataset_api = project.get_dataset_api()
dataset_api.upload("../resources/forecast.csv",
                   "Resources/weather_forecast",
                   overwrite=True # I do not want to display forecast of past days
                  )

Uploading: 0.000%|          | 0/1390 elapsed<00:00 remaining<?

'Resources/weather_forecast/forecast.csv'